# Glow-TTS: A Generative Flow for Text-to-Speech via Monotonic Alignment Search

- **Glow-TTS**: **dynamic programming**을 이용하여 **Text**로부터 **Mel-Spectrogram**을 생성하는 **flow 기반** 딥러닝 TTS 모델

# 0. Abstract

- **Glow-TTS**는 다음과 같은 특징을 지닌 모델이다.
    1. **Tacotron2**보다 **합성 속도**가 **15.7배 빠르다.**
    2. 모델을 **다중 화자(multi-speaker)로 확장**하기 쉽다.

# 1. Introduction

- **Glow-TTS**는 Text를 Mel-spectrogram으로 변환하는 모델이다.
<img src="01.png" width=70% height=70%>

- **Autoregressive Model**는 초반에 딥러닝 기반 TTS 모델 중 가장 성능이 좋았다.
    - 예) **Tacotron2**, **Transformer TTS**
    - **단점**
        1. **Undesirable Delay**: text 길이에 따라 추론 시간이 선형적으로 증가하다보니 real-time service에 어려움이 있음.
        2. **A lack of robustness**: Input에 **Repeated words**가 있으면 심각한 **Attention errors**가 발생하는 등의 문제 발생
        - 이를 극복하기 위해 **Parallel TTS Model**이 등장한다.

- **Parallel TTS Model**: 시간 순서에 관계없이 병렬로 음성을 생성하는 모델
    - 예) **FastSpeech**, AlignTTS, Flow-TTS
    - **장점**
        1. Autoregressive model보다 **상당히 빠른 Mel-spectrogram 합성 속도**
        2. monotonic alignment을 강제함으로써 다음과 같은 **failure cases**를 줄였다.  
            1) Mispronouncing(잘못된 발음), 2) skipping(생략), 3) Repeating words(반복된 단어)
    - **단점**: **External aligners의 성능에 크게 의존**한다는 문제가 있었다.
        - **External aligners(외부 정렬기)**라 불리는 **pre-trained autoregressive models**로부터 text와 speech사이의 **well-aligned attention maps**를 추출해야만 했다.

- **Glow-TTS**: **External aligner를 제거**한 **flow-based generative model(parallel TTS model의 일종)**
    1. **flows**와 **dynamic programming**을 이용해서 내부적으로 적절한 **monotonic alignment**을 탐색하고 강제한다.
        - 이는 **긴 발화를 생성하는 견고한 TTS 모델**을 만드는 데에 기여한다.
        - flows와 dynamic programming에 대한 설명은 이후에 언급한다.
    2. **flows**를 이용하여 **빠르고, 다양하고, 음성 합성 제어가 가능하게 한다.**
        - **Tacotron2보다 15.7배 빠른 합성 속도**를 갖는다.
        - **latent representation**을 변경함으로써 **다양한 억양 패턴을 합성**할 수 있고, **음 높이를 조정**할 수 있다.
        - 여기서 작은 수정만으로 **다중 화자** 지원이 가능하다.

# 2. Background

- 아래 내용은 논문에서 직접적으로 언급되지는 않지만, 논문의 내용을 이해하기 위해 필요한 배경 지식을 설명한다.

- **Flow-based generative model**: 일대일 변환 함수를 이용하여 입력 데이터와 잠재 변수 간의 확률 분포를 예측하는 모델
<img src="02.png" width=70% height=70%>
(출처: https://lilianweng.github.io/posts/2018-10-13-flow-models/)

    - **GAN(Generative Adversarial Networks)**: 생성자(generator)와 판별자(discriminator)의 경쟁적인 학습을 이용하는 딥러닝 생성 모델
    - **VAE(Variational Auto Encoder)**: 입력 데이터를 잠재 공간으로 압축한 다음, 잠재 공간에서 다시 원래 데이터로 복원하는 과정에서 간접적으로 데이터의 분포를 학습하여 새로운 데이터를 생성하는 딥러닝 생성 모델. Evidence lower bound (ELBO)를 손실함수로 사용한다.
    - **Flow-based generative model**: VAE와는 달리 명시적으로 데이터 분포를 학습하며, **가역 함수**를 이용하므로 데이터 손실이 적다.
    
    <img src="03.png" width=70% height=70%>
    (출처: https://lilianweng.github.io/posts/2018-10-13-flow-models/)
    
        - **flow 함수**는 가역 함수들의 합성 함수로 구성된다.
        - **잠재 공간**의 단순한 확률 분포에서 flow 함수를 거치면서 **실제 데이터의 복잡한 확률 분포**를 얻을 수 있도록 학습한다.

- **Dynamic Programming(동적 계획법)**: 특정 문제를 풀기 위해 다른 문제의 답을 기억하면서 이용하는 방법
<img src="04.png" width=70% height=70%>

    - 위 그림과 같이 정의된 피보나치 수열을 구하려고 한다. 즉, F(4)를 구하려고 한다.
        - **재귀적 방법(Recursive Algorithm)**은 F(4)를 구하기 위해 **4번 연산**해야 한다.
        - 그러나 **F(2)를 기억하는 동적 계획법(Dynamic Programming)**을 이용하면 F(2)를 다시 계산하지 않아도 되므로 **3번만 연산**하면 된다.
    - 더 많은 재귀 호출이 일어날 수록 dynamic programming과 기하급수적으로 연산량에 차이를 보인다.
    
- Dynamic Programming의 두 가지 방식
    - **Top-down**: F(4) -> F(3) -> F(2) 순서로 계산
    - **Bottom-up**: F(2) -> F(3) -> F(4) 순서로 계산

# 3. Architecture

## 3.1. Training and Inference Procedures

### 3.1.1. Training Procedures

- **Glow-TTS**의 **훈련 절차**는 다음과 같다.
<img src="05.png" width=50% height=50%>

    - $T_{text}$: the length of the text input, $T_{mel}$: the length of the input mel-spectrogram
    - $c = c_{1:T_{text}}$: text, $z = z_{1:T_{mel}}$: latent representation, $x = x_{1:T_{mel}}$: mel-spectrogram
    - $P_{Z}(z|c)$: $c$에 대한 $z$의 확률 분포, $P_{X}(x|c)$: $c$에 대한 $x$의 확률 분포
    - $A$: alignment function, $f_{enc}(c)=(\mu, \sigma)$, $f_{dec}(z)=x$라 하자. 이때 $f_{dec}$는 **가역 함수**, $A$는 **단조(monotonic) 전사(surjective)함수**라 가정한다.
    - 사전 확률 분포 $P_{Z}$는 **isotropic multivariate Gaussian distribution(공분산 행렬이 항등행렬인 다변량 정규분포)**라 가정하자.
        - $z=(z_{1}, \dots, z_{T_{text}})$는 공분산 행렬이 항등행렬이고 각 변수가 정규분포이므로 서로 **mutually independent**이다.

- 이제 Training을 위한 식을 전개해보자. (논문에는 중간 과정이 생략되어 있다.)
    - 변수 변환(Change of variable formula)에 의해 다음 식이 성립한다.
    $$ P_{X}(x|c) = P_{Z}(f^{-1}(x)|c)\det \left( \frac{\partial f^{-1}(x)}{\partial x} \right) $$
    - $f^{-1}(x)=z$이므로 양변에 로그를 취하면 다음과 같다.
    $$ \log P_{X}(x|c) = \log P_{Z}(z|c) + \log \left| \det \left( \frac{\partial f^{-1}(x)}{\partial x} \right) \right| \cdots(1)$$
    - $z_{j} \sim N(z_{j};\mu_{i}, \sigma_{i})$이면 $A(j) = i$이라 하자. 그러면 다음 식이 성립한다.
    $$ P_{z_{j}}(z_{j}|c;\theta, A) = N(z_{j};\mu_{A(j)}, \sigma_{A(j)}) \cdots(2)$$
    - $z$는 각 변수에 대해 독립이므로 다음 식이 성립한다.
    $$ P_{z}(z|c;\theta, A) = P_{z_{1}}(z_{1}|c;\theta, A)P_{z_{2}}(z_{2}|c;\theta, A) \cdots P_{z_{T_{mel}}}(z_{T_{mel}}|c;\theta, A) = \prod_{j=1}^{T_{mel}} P_{z_{j}}(z_{j}|c;\theta, A) \cdots (3)$$
    - 식 (3)에 식 (2)을 적용하고 양변에 로그를 취하면 다음과 같다.
    $$ \log P_{Z}(z|c;\theta, A) = \sum_{j=1}^{T_{mel}} \log N(z_{j};\mu_{A(j)}, \sigma_{A(j)}) \cdots (4)$$

- 우리는 $P_{X}(x|c)$가 최대가 되길 원한다. 즉, log-likelihood $L(\theta, A) = \log P_{X}(x|c;A, \theta)$가 최대가 되는 $\theta$와 $A$를 찾고 싶다.
    - 그러나 $\max_{\theta, A} L(\theta, A) = \max_{\theta, A} \log P_{X}(x|c;A, \theta)$의 global solution을 찾는 것은 계산적으로 매우 어렵다.
    - 그래서 이를 구하는 과정을 두 단계로 나눈다.
        1. 현재 $\theta$에 대한 가장 적절한 $A^{*}$를 찾는다.
            - 식 (1)와 식(4)에 의해 식은 다음과 같이 정리된다.
            $$ A^{*}=\arg\max_{A} \log P_{X}(x|c;A, \theta) = \arg\max_{A} \sum_{j=1}^{T_{mel}} \log N(z_{j}; \mu_{A(j)}, \sigma_{A(j)}) \cdots (5)$$
        2. $\log P_{X}(x|c;\theta, A^{*})$를 최대화하도록 $\theta$를 업데이트한다.
    - 위 1번 과정은 이후에 소개할 **Monotonic Alignment Search(MAS) Algorithm**으로 해결할 수 있다.

- **Duration Predictor**: 더 적절한 alignment $A^{*}$를 추정하기 위해서 **각 text에 대응되는 alignment의 길이(duration)를 예측**하는 함수
    - $f_{dur}$: Duration predictor, $sg[\cdot]$: stop gradient operator(gradient를 backward 전파하지 않도록 함.)이라 하자.
    - 이때 Duration predictor의 Loss는 다음과 같이 설정한다.
    $$d_{i} = \sum_{j=1}^{T_{mel}} 1_{A*(j)=i}, \, i = 1, \cdots, T_{text}$$
    $$L_{dur} = MSE(f_{dur}(sg[f_{enc}(c)]), d)$$

### 3.1.2. Inference Procedure

- **Glow-TTS**의 **추론 절차**는 다음과 같다.
<img src="06.png" width=50% height=50%>

    - Alignment는 **Duration Predictor**의 결과로부터 Ceil(올림)하여 결정한다.
    - latent representation은 Encoder로부터 추론된 prior distribution으로부터 sampling한다.
    - Mel-spectrogram은 flow-based Decoder로부터 생성된다.

## 3.2. Monotonic Alignment Search

- 다음은 Monotonic Alignment의 예시이다.
<img src="07.png" width=50% height=50%>

- $Q_{i, j}$를 $i$번째 prior distribution와 $j$번째 latent variable까지 주어졌을 때의 **최대 로그 우도**라 하자.
<img src="08.png" width=50% height=50%>
    
- **(Step 1)**
    - 그러면 $z_{j}$의 이전 latent variable은 $z_{j-1}$이고, $A$의 **monotonicity**와 **surjection**에 의해 $z_{j-1}$은 $(\mu_{i-1}, \sigma_{i-1})$ 또는 $(\mu_{i}, \sigma_{i})$ 에 의해 정렬된다.
    - 따라서 식(5)에 의해 $Q_{i, j}$가 다음과 같이 재귀적으로 결정된다.
    $$Q_{i, j} = \max_{A} \sum_{k=1}^{j} \log N(z_{k}; \mu_{A(k)}, \sigma_{A(k)})=\max(Q_{i-1, j-1}, Q_{i, j-1})+\log N(z_{j}; \mu_{i}, \sigma_{i})$$
    - 이러한 방식으로 $Q_{T_{text}, T_{mel}}$까지 계산할 수 있다.  
    
- **(Step 2)**
    - 그 다음 $A^{*}$를 결정하기 위해서 **Backtracking**을 진행한다.
    <img src="09.png" width=60% height=60%>
    
    - $A^{*}(T_{mel})=T_{text}$로 둔 다음에 backtracking하여 Alignment를 추정해간다.

- 그런데 이를 재귀적인 방법으로 시도하면 상당히 많은 overhead가 발생한다!
- 위 알고리즘에 **Dynamic Programming**을 적용한 것이 **Monotonic Alignmnet Search Algorithm**이다.
<img src="10.png" width=70% height=70%>

- 알고리즘을 그림으로 정리해보자.
<img src="11.png" width=50% height=50%>
<img src="12.png" width=50% height=50%>
<img src="13.png" width=50% height=50%>

    - 이후에 Backtracking을 진행한다.

## 3.3. Background for Model Architecture

- Glow-TTS 모델의 구조를 이해하기 위해서는 다음을 배경으로 알아두어야 한다.
    1. **Transformer**
    2. **Relative Position Representations**
    3. **Glow**

### 3.3.1. Transformer

- **Transformer**: CNN, RNN과 같은 구조 없이 오로지 **Self Attention** 구조만을 활용하여 여러 딥러닝 분야에서 우수한 성능을 보여주는 모델

<img src="14.png" width=70% height=70%>

- **Self Attention**: 하나의 데이터를 Query, Key, Value에 입력함으로써 **자기 자신의 상호 위치 관계**를 학습하는 Attention 구조
    - 기존의 Attention에서는 다음과 같이 입력을 설정하여 Decoder의 입력과 Encoder의 입력 사이의 위치 관계를 학습한다.
        - Query: Decoder의 hidden state(feature)
        - Key, Value: Encoder의 hidden states(features)
- **Multi-Head Attention**: channel 차원을 기준으로 여러 개의 헤드(head)로 분할하여 각각 다른 어텐션 구조를 거치고, 그 결과를 결합하는 구조
    - 이 방식을 활용하면 서로 다른 어텐션 구조가 다양한 위치 관계를 학습하여 더 높은 성능을 보여준다.
- **Positional Encoding**: 모델에게 입력 데이터의 위치 정보를 제공하기 위한 기법
    - Transformer 모델은 CNN과 RNN같은 구조가 사용되지 않기 때문에 Positional Encoding이 없다면 입력 데이터의 위치 정보를 알 수 없다.
    - Positional Encoding은 다음과 같은 사인함수와 코사인함수 값을 기존에 Embedding된 입력 데이터에 더해줌으로써 위치 정보를 부여한다.
    $$PE_{(pos, 2i)} = \sin(pos/10000^{2i/d})$$
    $$PE_{(pos, 2i+1)} = \cos(pos/10000^{2i/d})$$
    ($pos$: position, $i$: dimension, $d$: total dimension)

### 3.3.2. Relative Position Representations

- **Relative Position Representations**: 모델에게 입력 데이터의 절대적인 위치 정보가 아닌, **상대적인 위치 정보**를 제공하기 위한 기법
    - 기존의 Self Attention은 다음과 같이 계산된다.
    $$z_{i} = \sum_{j=1}^{n} \alpha_{ij}(x_{j}W^{V})$$
    $$\alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{n} \exp(e_{ik})}$$
    $$e_{ij} = \frac{(x_{i}W^{Q})(x_{j}W^{K})^{T}}{\sqrt{d_{z}}}$$
    ($z_{i}$: output, $\alpha_{ij}$: attention weight, $e_{ij}$: score, $x_{i}$: input(query), $d_{z}$: output dimension)
    - Relative Position Representations를 적용하면 다음과 같이 계산된다.
    $$z_{i} = \sum_{j=1}^{n} \alpha_{ij}(x_{j}W^{V} + a_{ij}^{V})$$
    $$\alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{n} \exp(e_{ik})}$$
    $$e_{ij} = \frac{(x_{i}W^{Q})(x_{j}W^{K} + a_{ij}^{K})^{T}}{\sqrt{d_{z}}}$$
    ($a_{ij}^{K}, a_{ij}^{V}$: distinct edge representation, 이 representation이 상대적인 위치 정보를 전달하는 역할을 한다.)
    
    <img src="15.png" width=50% height=50%>
    
    - 위 그림과 같이 $a_{ij}$는 기준이 되는 벡터로부터의 거리를 반영한다.

### 3.3.3. Glow

- **Glow**: ActNorm, 1x1 invertible convolution, Affine coupling layer를 활용한 flow 기반 모델
<img src="16.png" width=30% height=30%>

    - **ActNorm**: Batch Normalization을 대체한 Activation Normalization으로, BatchNorm의 단점을 보완한 정규화 방법
        - BatchNorm은 배치에 대하여 정규화하기 때문에 배치 크기가 작을 수록 평균과 표준편차를 추정하는 데에 어려움이 있다. 이러 인해 노이즈가 커져 성능이 크게 저하된다.
    
    - **1x1 Invertible Convolution**: 가역인 가중치 행렬을 이용하여 데이터를 shuffle해주는 역할을 하는 합성곱 연산
        - 이후에 나오는 Affine coupling layer에서 앞쪽 데이터가 섞이지 않는다. 이를 위해 1x1 Invertible Conv에서 shuffle을 한다.
        - 나중에 log likelihood를 구하기 위해서는 각 가중치 행렬의 determinant를 구할 수 있어야 한다.
        - Determinant를 구하는 복잡도를 $O(c^{3})$에서 $O(c)$로 줄이기 위해 QR decomposition을 이용하여 직교 행렬을 가중치 행렬로 사용한다.
    <img src="17.png" width=70% height=70%>
    
        - NN: Nonlinear mapping
    
    - **Affine Coupling Layer**: 가중치 행렬의 determinant를 구하기 쉬운 Affine layer의 변형
        - 다음 그림과 같이 데이터를 둘로 나눈 뒤에 간단한 연산을 취하기 때문에 determinant를 구하기 쉽다.
        <img src="18.png" width=70% height=70%>

## 3.4. Model Architecture

### 3.4.1. Encoder

- **Encoder**의 전체 구조는 다음과 같다.
<img src="19.png" width=50% height=50%>

- **Multi-Head Attention**은 **Self Attention**구조를 가지며, Positional Encoding 대신 **Relative Position Representations**가 적용되었다.
- **Duration Predictor**는 FastSpeech에서 제안된 구조와 같다.

### 3.4.2. Decoder

- **Decoder**의 전체 구조는 다음과 같다.
<img src="20.png" width=70% height=70%>

- (왼쪽) Decoder의 전체 구조
- (가운데) **Squeeze**와 **Unsqueeze**: 계산 효율성을 위해 80개의 채널을 가진 mel-spectrogram을 160개의 채널을 가지도록 shape를 조절해준다.
- (오른쪽) **Invertible 1x1 Conv**: Jacobian determinant 계산 속도를 감소시키기 위해 1x1 conv 전에 순서를 조정해준다.
- (아래) **Affine Coupling Layer**는 아래 그림의 WaveGlow 구조를 따라간다.
<img src="21.png" width=50% height=50%>

    - **WN**: WaveNet과 Parallel WaveNet의 구조와 유사한 trasformation. (다른 transformation으로 대체되어도 상관 없다.)

**[참고 자료]**
1. Glow-TTS paper: https://arxiv.org/abs/2005.11129
2. Flow-based generative model github blog: https://lilianweng.github.io/posts/2018-10-13-flow-models/
3. Flow-based generative model tistory blog: https://sanghyu.tistory.com/18
4. Multi-Head Attention paper: https://arxiv.org/abs/1706.03762
5. Positional Encoding blog: https://www.blossominkyung.com/deeplearning/transfomer-positional-encoding
6. Relative Position Representations paper: https://arxiv.org/abs/1803.02155
7. Real NVP (Coupling Layer에 대한 설명 포함) paper: https://arxiv.org/abs/1605.08803
8. Glow blog1: https://sanghyu.tistory.com/18
9. Glow paper: https://arxiv.org/abs/1807.03039
10. WaveGlow paper: https://arxiv.org/abs/1811.00002